<a href="https://colab.research.google.com/github/PaulMadhusmita/GestureRecognition/blob/main/Gesture_Recognition_Madhusmita_Paul_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import imageio
from imageio import imread
import cv2
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# Note: the zip location needs to be updated below accordingly
%cd '/content/gdrive/MyDrive'

Mounted at /content/gdrive
/content/gdrive/MyDrive


In [4]:
%%shell
if [ ! -f Project_data.zip ]; then
    wget 'https://docs.google.com/uc?export=download&id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL&confirm=t' -O Project_data.zip
fi

In [5]:
# Note :Provide Correct zip file name. Checking and clearing 'Skin cancer ISIC The International Skin Imaging Collaboration' folder,
# to refresh dataset, to prevent data tampering.
zipFileName= 'Project_data.zip'
datasetFolderName='Project_data'
if not os.path.exists(datasetFolderName):
  !unzip  "$zipFileName"
else:
  !rm -rf "$datasetFolderName"
  !unzip  "$zipFileName"

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

In [6]:
!pwd


/content/gdrive/MyDrive


In [7]:
train_doc = np.random.permutation(open("Project_data/train.csv").readlines())
val_doc = np.random.permutation(open("Project_data/val.csv").readlines())
batch_size = 20 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [8]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

In [9]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = cv2.resize(image,[120,120])
                    image = image - np.percentile(image,5)/ np.percentile(image,95) - np.percentile(image,5)
                    batch_data[folder,idx,:,:,0] = image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    image = cv2.resize(image,[120,120])
                    image = image - np.percentile(image,5)/ np.percentile(image,95) - np.percentile(image,5)
                    batch_data[folder,idx,:,:,0] = image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] #normalise and feed in the image
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels        


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


# Model

Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout, GlobalAveragePooling2D, GlobalAveragePooling3D, ConvLSTM2D
from keras.layers.convolutional import Conv2D, Conv3D, MaxPooling3D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [30]:
model0 = Sequential()

model0.add(Conv3D(16, kernel_size=(3,3,3), input_shape=(30,120,120,3), activation='relu', padding='same'))
model0.add(MaxPooling3D(pool_size=(1,2,2)))

model0.add(Flatten())

model0.add(Dense(512, activation='relu'))
model0.add(Dense(5, activation='softmax'))

ResourceExhaustedError: ignored

### Model1

Adding more layers

In [14]:
#write your model here
model1 = Sequential()

model1.add(Conv3D(16, kernel_size=(3,3,3), input_shape=(30,120,120,3), activation='relu', padding='same'))
model1.add(MaxPooling3D(pool_size=(2,2,2)))
model1.add(Conv3D(32, kernel_size=(1,3,3), activation='relu', padding='same'))
model1.add(MaxPooling3D(pool_size=(2,2,2)))
model1.add(Conv3D(64, kernel_size=(1,3,3), activation = 'relu', padding='same'))
model1.add(MaxPooling3D(pool_size=(2,2,2)))

model1.add(Flatten())

model1.add(Dense(128, activation='relu'))
model1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [15]:
optimiser = optimizers.Adam(lr=0.01)
model1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 30, 120, 120, 16)  1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 60, 60, 16)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 15, 60, 60, 32)    4640      
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 7, 30, 30, 32)    0         
 3D)                                                             
                                                                 
 conv3d_2 (Conv3D)           (None, 7, 30, 30, 64)     18496     
                                                                 
 max_pooling3d_2 (MaxPooling  (None, 3, 15, 15, 64)    0

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [16]:
train_generator = generator(train_path, train_doc, batch_size) # batch_size = 20
val_generator = generator(val_path, val_doc, batch_size)

In [15]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [16]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [19]:
model1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 222.2294 - categorical_accuracy: 0.2142 Source path =  Project_data/val ; batch size = 20

Epoch 1: saving model to model_init_2023-01-1213_26_52.296083/model-00001-222.22945-0.21418-1.61281-0.21000.h5
34/34 [==============================] - 1178s 34s/step - loss: 222.2294 - categorical_accuracy: 0.2142 - val_loss: 1.6128 - val_categorical_accuracy: 0.2100 - lr: 0.0100
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 1.6081 - categorical_accuracy: 0.1863
Epoch 2: saving model to model_init_2023-01-1213_26_52.296083/model-00002-1.60810-0.18627-1.60056-0.21000.h5
34/34 [==============================] - 227s 7s/step - loss: 1.6081 - categorical_accuracy: 0.1863 - val_loss: 1.6006 - val_categorical_accuracy: 0.2100 - lr: 0.0100
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 1.6236 - categorical_accuracy: 0.1863
Epoch 3: saving mod

### Model2 

Changing the Batch_size to 30


In [37]:
# Creating another model by adding more layers and changing Batch_size

model2 = Sequential()

model2.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=(30,120,120,3)))
model2.add(Conv3D(64, kernel_size=3, activation='relu'))
model2.add(MaxPooling3D(pool_size=(3, 3, 3)))

model2.add(Conv3D(64, kernel_size=3, activation='relu'))
model2.add(Conv3D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling3D(pool_size=(3, 3, 3)))

model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(Dense(5, activation='softmax'))

In [38]:
optimiser = optimizers.Adam()
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_44 (Conv3D)          (None, 28, 118, 118, 32)  2624      
                                                                 
 conv3d_45 (Conv3D)          (None, 26, 116, 116, 64)  55360     
                                                                 
 max_pooling3d_22 (MaxPoolin  (None, 8, 38, 38, 64)    0         
 g3D)                                                            
                                                                 
 conv3d_46 (Conv3D)          (None, 6, 36, 36, 64)     110656    
                                                                 
 conv3d_47 (Conv3D)          (None, 4, 34, 34, 128)    221312    
                                                                 
 max_pooling3d_23 (MaxPoolin  (None, 1, 11, 11, 128)   0         
 g3D)                                                

In [39]:
train_generator = generator(train_path, train_doc, 30) # changing the batch_size to 30
val_generator = generator(val_path, val_doc, 30)

In [40]:
model2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 30
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 30.5248 - categorical_accuracy: 0.2170  Source path =  Project_data/val ; batch size = 30
Batch:  4 Index: 30

Epoch 1: saving model to model_init_2023-01-1306_37_16.392607/model-00001-30.52476-0.21695-1.63203-0.20909.h5
34/34 [==============================] - 4719s 137s/step - loss: 30.5248 - categorical_accuracy: 0.2170 - val_loss: 1.6320 - val_categorical_accuracy: 0.2091 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 1.5828 - categorical_accuracy: 0.2549 
Epoch 2: saving model to model_init_2023-01-1306_37_16.392607/model-00002-1.58277-0.25490-1.49962-0.36000.h5
34/34 [==============================] - 728s 21s/step - loss: 1.5828 - categorical_accuracy: 0.2549 - val_loss: 1.4996 - val_categorical_accuracy: 0.3600 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 1.4999 - categorical_accuracy: 0.343

In [ ]:
# Model 3

# Changing batch_size = 36 and adding Batchnormalization to the layers

In [46]:
# Creating another model by adding more layers and changing Batch_size

model3 = Sequential()

model3.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=(30,120,120,3)))
model3.add(MaxPooling3D(pool_size=(2, 2, 2)))
model3.add(BatchNormalization())

model3.add(Conv3D(64, kernel_size=3, activation='relu'))
model3.add(Conv3D(128, kernel_size=3, activation='relu'))
model3.add(MaxPooling3D(pool_size=(2, 2, 2)))
model3.add(BatchNormalization())

model3.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))

model3.add(MaxPooling3D(pool_size=(2, 2, 2)))
model3.add(BatchNormalization())

model3.add(Flatten())
model3.add(Dense(512, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dense(5, activation='softmax'))

In [47]:
optimiser = optimizers.Adam()
model3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model3.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_43 (Conv3D)          (None, 28, 118, 118, 32)  2624      
                                                                 
 max_pooling3d_33 (MaxPoolin  (None, 14, 59, 59, 32)   0         
 g3D)                                                            
                                                                 
 batch_normalization_44 (Bat  (None, 14, 59, 59, 32)   128       
 chNormalization)                                                
                                                                 
 conv3d_44 (Conv3D)          (None, 12, 57, 57, 64)    55360     
                                                                 
 conv3d_45 (Conv3D)          (None, 10, 55, 55, 128)   221312    
                                                                 
 max_pooling3d_34 (MaxPoolin  (None, 5, 27, 27, 128) 

In [48]:
train_generator = generator(train_path, train_doc, 36)
val_generator = generator(val_path, val_doc, 36)

In [49]:
model3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)



Source path =  Project_data/train ; batch size = 36
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.6016 - categorical_accuracy: 0.6239  Source path =  Project_data/val ; batch size = 36
Batch:  3 Index: 36

Epoch 1: saving model to model_init_2023-01-1705_24_57.997314/model-00001-1.60158-0.62387-254.92554-0.23718.h5
34/34 [==============================] - 3667s 107s/step - loss: 1.6016 - categorical_accuracy: 0.6239 - val_loss: 254.9255 - val_categorical_accuracy: 0.2372 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.5009 - categorical_accuracy: 0.8311 Batch:  4 Index: 28

Epoch 2: saving model to model_init_2023-01-1705_24_57.997314/model-00002-0.50090-0.83111-21.62176-0.40000.h5
34/34 [==============================] - 1841s 54s/step - loss: 0.5009 - categorical_accuracy: 0.8311 - val_loss: 21.6218 - val_categorical_accuracy: 0.4000 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 1.9056 - cat

Creating another generator function with input size 50

In [34]:
x = 30 # number of frames
y = 50 # image width
z = 50 # image height

In [35]:
def generator_new(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = cv2.resize(image,[y,z])
                    image = image - np.percentile(image,5)/ np.percentile(image,95) - np.percentile(image,5)
                    batch_data[folder,idx,:,:,0] = image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,len(img_idx),y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    image = cv2.resize(image,[y,z])
                    image = image - np.percentile(image,5)/ np.percentile(image,95) - np.percentile(image,5)
                    batch_data[folder,idx,:,:,0] = image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] #normalise and feed in the image
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels        



### Model 4

Increasing the Batch_size to 40 and reducing image size to 50X50 and adding Dropout layers 

In [11]:
model4 = Sequential()

model4.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=(30,50,50,3)))
model4.add(Conv3D(64, kernel_size=3, activation='relu'))
model4.add(MaxPooling3D(pool_size=(2, 2, 2)))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))

model4.add(Conv3D(128, kernel_size=3, activation='relu'))
model4.add(MaxPooling3D(pool_size=(2, 2, 2)))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))

model4.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
model4.add(MaxPooling3D(pool_size=(2, 2, 2)))
model4.add(BatchNormalization())
model4.add(Dropout(0.2))

model4.add(Flatten())
model4.add(Dense(512, activation='relu'))
model4.add(BatchNormalization())
model4.add(Dense(5, activation='softmax'))

In [14]:
optimiser = optimizers.Adam()
model4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model4.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 48, 48, 32)    2624      
                                                                 
 conv3d_1 (Conv3D)           (None, 26, 46, 46, 64)    55360     
                                                                 
 max_pooling3d (MaxPooling3D  (None, 13, 23, 23, 64)   0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 13, 23, 23, 64)   256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 13, 23, 23, 64)    0         
                                                                 
 conv3d_2 (Conv3D)           (None, 11, 21, 21, 128)   2

In [15]:
train_generator = generator_new(train_path, train_doc, 40)
val_generator = generator_new(val_path, val_doc, 40)

In [18]:
model4.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 40
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.6919 - categorical_accuracy: 0.5882 Source path =  Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 1: saving model to model_init_2023-01-1712_30_59.621302/model-00001-1.69187-0.58824-42.96287-0.20000.h5
34/34 [==============================] - 1513s 44s/step - loss: 1.6919 - categorical_accuracy: 0.5882 - val_loss: 42.9629 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.5246 - categorical_accuracy: 0.8014 
Epoch 2: saving model to model_init_2023-01-1712_30_59.621302/model-00002-0.52458-0.80145-8.65836-0.30000.h5
34/34 [==============================] - 970s 28s/step - loss: 0.5246 - categorical_accuracy: 0.8014 - val_loss: 8.6584 - val_categorical_accuracy: 0.3000 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.4546 - categorical_accuracy: 0.8322 


### Model5

Adding more Dropout Layers to reduce overfitting and increasing Batch_size to 46

In [12]:
model5 = Sequential()

model5.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=(30,50,50,3)))
model5.add(MaxPooling3D(pool_size=(2, 2, 2)))
model5.add(BatchNormalization())
model5.add(Dropout(0.25))

model5.add(Conv3D(64, kernel_size=3, activation='relu'))
model5.add(Conv3D(128, kernel_size=3, activation='relu'))
model5.add(MaxPooling3D(pool_size=(2, 2, 2)))
model5.add(BatchNormalization())
model5.add(Dropout(0.25))
model5.add(Dropout(0.5))

model5.add(Conv3D(128, kernel_size=(1, 3, 3), activation='relu'))
model5.add(MaxPooling3D(pool_size=(1, 2, 2)))
model5.add(BatchNormalization())
model5.add(Dropout(0.25))

model5.add(Flatten())
model5.add(Dense(256, activation='relu'))
model5.add(Dropout(0.25))
model5.add(BatchNormalization())
model5.add(Dense(5, activation='softmax'))

In [13]:
optimiser = optimizers.Adam()
model5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model5.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 48, 48, 32)    2624      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 14, 24, 24, 32)   0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 14, 24, 24, 32)   128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 14, 24, 24, 32)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 22, 22, 64)    55360     
                                                                 
 conv3d_2 (Conv3D)           (None, 10, 20, 20, 128)   2

In [14]:
train_generator = generator_new(train_path, train_doc, 46)
val_generator = generator_new(val_path, val_doc, 46)

In [17]:
model5.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 46
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.3110 - categorical_accuracy: 0.5664Source path =  Project_data/val ; batch size = 46
Batch:  3 Index: 46

Epoch 1: saving model to model_init_2023-01-1802_06_55.791550/model-00001-1.31099-0.56641-19.73254-0.25000.h5
34/34 [==============================] - 169s 5s/step - loss: 1.3110 - categorical_accuracy: 0.5664 - val_loss: 19.7325 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.6497 - categorical_accuracy: 0.7747
Epoch 2: saving model to model_init_2023-01-1802_06_55.791550/model-00002-0.64966-0.77467-3.56673-0.65000.h5
34/34 [==============================] - 97s 3s/step - loss: 0.6497 - categorical_accuracy: 0.7747 - val_loss: 3.5667 - val_categorical_accuracy: 0.6500 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.5330 - categorical_accuracy: 0.7907Batch:  

### Model6 - TimeDistributed Conv2D + GRU

In [25]:
model6 = Sequential()
model6.add(TimeDistributed(
     Conv2D(32, (3,3), activation='relu'), input_shape=(30,50,50,3))
 )
model6.add(TimeDistributed(
         MaxPooling2D((2,2)))
 )
model6.add(BatchNormalization())
model6.add(Dropout(0.25))

model6.add(TimeDistributed(
     Conv2D(64, (3,3), activation='relu'))
 )
model6.add(TimeDistributed(
     MaxPooling2D((2,2)))
 )
model6.add(BatchNormalization())
model6.add(Dropout(0.25))

model6.add(TimeDistributed(GlobalAveragePooling2D()))
model6.add(TimeDistributed(Dense(64, activation='relu')))
model6.add(BatchNormalization())
model6.add(Dropout(0.25))

model6.add(GRU(128))
model6.add(BatchNormalization())
model6.add(Dense(5, activation='softmax'))

In [26]:
optimiser = optimizers.Adam()
model6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model6.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_4 (TimeDis  (None, 30, 48, 48, 32)   896       
 tributed)                                                       
                                                                 
 time_distributed_5 (TimeDis  (None, 30, 24, 24, 32)   0         
 tributed)                                                       
                                                                 
 batch_normalization_6 (Batc  (None, 30, 24, 24, 32)   128       
 hNormalization)                                                 
                                                                 
 dropout_7 (Dropout)         (None, 30, 24, 24, 32)    0         
                                                                 
 time_distributed_6 (TimeDis  (None, 30, 22, 22, 64)   18496     
 tributed)                                            

In [27]:
train_generator = generator_new(train_path, train_doc, 64)
val_generator = generator_new(val_path, val_doc, 64)

In [28]:
model6.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 64
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.3666 - categorical_accuracy: 0.4622Source path =  Project_data/val ; batch size = 64
Batch:  2 Index: 64
Batch:  3 Index: 36

Epoch 1: saving model to model_init_2023-01-1802_06_55.791550/model-00001-1.36663-0.46225-2.00773-0.23500.h5
34/34 [==============================] - 197s 6s/step - loss: 1.3666 - categorical_accuracy: 0.4622 - val_loss: 2.0077 - val_categorical_accuracy: 0.2350 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.9841 - categorical_accuracy: 0.5961Batch:  4 Index: 28

Epoch 2: saving model to model_init_2023-01-1802_06_55.791550/model-00002-0.98409-0.59610-1.59665-0.28846.h5
34/34 [==============================] - 106s 3s/step - loss: 0.9841 - categorical_accuracy: 0.5961 - val_loss: 1.5967 - val_categorical_accuracy: 0.2885 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.8827

### Model7 - TimeDistributed Conv2D + Dense



In [29]:
model7 = Sequential()
model7.add(TimeDistributed(
    Conv2D(32, (3,3), activation='relu'), input_shape=(30,50,50,3))
 )
model7.add(TimeDistributed(
    MaxPooling2D((2,2)))
 )
model7.add(BatchNormalization())
model7.add(Dropout(0.25))

model7.add(TimeDistributed(
    Conv2D(64, (3,3), activation='relu'))
 )
model7.add(TimeDistributed(
    MaxPooling2D((2,2)))
 )
model7.add(BatchNormalization())
model7.add(Dropout(0.25))

model7.add(TimeDistributed(
    Conv2D(128, (3,3), activation='relu'))
 )
model7.add(TimeDistributed(
    MaxPooling2D((2,2)))
 )
model7.add(BatchNormalization())
model7.add(Dropout(0.5))

model7.add(GlobalAveragePooling3D())
model7.add(Dense(256, activation='relu'))
model7.add(BatchNormalization())
model7.add(Dense(5, activation='softmax'))

In [30]:
optimiser = optimizers.Adam()
model7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model7.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_10 (TimeDi  (None, 30, 48, 48, 32)   896       
 stributed)                                                      
                                                                 
 time_distributed_11 (TimeDi  (None, 30, 24, 24, 32)   0         
 stributed)                                                      
                                                                 
 batch_normalization_10 (Bat  (None, 30, 24, 24, 32)   128       
 chNormalization)                                                
                                                                 
 dropout_10 (Dropout)        (None, 30, 24, 24, 32)    0         
                                                                 
 time_distributed_12 (TimeDi  (None, 30, 22, 22, 64)   18496     
 stributed)                                           

In [32]:
train_generator = generator_new(train_path, train_doc, 72)
val_generator = generator_new(val_path, val_doc, 72)

In [33]:
model7.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 72
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.1342 - categorical_accuracy: 0.5836Source path =  Project_data/val ; batch size = 72
Batch:  2 Index: 72
Batch:  4 Index: 28

Epoch 1: saving model to model_init_2023-01-1802_06_55.791550/model-00001-1.13424-0.58358-1.70434-0.29891.h5
34/34 [==============================] - 170s 5s/step - loss: 1.1342 - categorical_accuracy: 0.5836 - val_loss: 1.7043 - val_categorical_accuracy: 0.2989 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 1.0837 - categorical_accuracy: 0.5673
Epoch 2: saving model to model_init_2023-01-1802_06_55.791550/model-00002-1.08365-0.56725-4.43350-0.26250.h5
34/34 [==============================] - 64s 2s/step - loss: 1.0837 - categorical_accuracy: 0.5673 - val_loss: 4.4335 - val_categorical_accuracy: 0.2625 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 1.6226 - categorical_accura

## Final Model
### Model8 - TimeDistributed + ConvLSTM2D

In [35]:
model8 = Sequential()
model8.add(TimeDistributed(
    Conv2D(8, (3,3), activation='relu'), input_shape=(30,50,50,3))
 )
model8.add(BatchNormalization())
model8.add(TimeDistributed(
    Conv2D(16, (3,3), activation='relu'))
 )
model8.add(BatchNormalization())
model8.add(
    ConvLSTM2D(8, kernel_size = 3, return_sequences=False)
 )
model8.add(BatchNormalization())
model8.add(TimeDistributed(
    Dense(64, activation='relu'))
 )
model8.add(BatchNormalization())
model8.add(GlobalAveragePooling2D())
model8.add(Dense(64, activation='relu'))
model8.add(Dense(5, activation='softmax'))

In [36]:
optimiser = optimizers.Adam()
model8.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model8.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_16 (TimeDi  (None, 30, 48, 48, 8)    224       
 stributed)                                                      
                                                                 
 batch_normalization_14 (Bat  (None, 30, 48, 48, 8)    32        
 chNormalization)                                                
                                                                 
 time_distributed_17 (TimeDi  (None, 30, 46, 46, 16)   1168      
 stributed)                                                      
                                                                 
 batch_normalization_15 (Bat  (None, 30, 46, 46, 16)   64        
 chNormalization)                                                
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 44, 44, 8)        

In [37]:
train_generator = generator_new(train_path, train_doc, 64)
val_generator = generator_new(val_path, val_doc, 64)

In [38]:
model8.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=20, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/train ; batch size = 64
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 1.4531 - categorical_accuracy: 0.4094Source path =  Project_data/val ; batch size = 64
Batch:  2 Index: 64
Batch:  3 Index: 36

Epoch 1: saving model to model_init_2023-01-1802_06_55.791550/model-00001-1.45307-0.40940-1.60175-0.23500.h5
34/34 [==============================] - 207s 6s/step - loss: 1.4531 - categorical_accuracy: 0.4094 - val_loss: 1.6018 - val_categorical_accuracy: 0.2350 - lr: 0.0010
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 1.0810 - categorical_accuracy: 0.5751Batch:  4 Index: 28

Epoch 2: saving model to model_init_2023-01-1802_06_55.791550/model-00002-1.08098-0.57508-1.54728-0.25962.h5
34/34 [==============================] - 118s 4s/step - loss: 1.0810 - categorical_accuracy: 0.5751 - val_loss: 1.5473 - val_categorical_accuracy: 0.2596 - lr: 0.0010
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.8427